# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 5
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [6]:
train_paths = glob(r'C:\Users\LG\Deep_Learning_Study\Tensorflow\dataset\cifar\train\*.png')[:100]
test_paths = glob(r'C:\Users\LG\Deep_Learning_Study\Tensorflow\dataset\cifar\test\*.png')[:100]

In [7]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [8]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [9]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [10]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [11]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [12]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [14]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Training

http://localhost:6006

In [15]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs
)

Epoch 1/5
3/3 [==============================] - 1s 395ms/step - loss: 2.4092 - accuracy: 0.0588 - val_loss: 2.3039 - val_accuracy: 0.0729
Epoch 2/5
3/3 [==============================] - 1s 313ms/step - loss: 2.3516 - accuracy: 0.0882 - val_loss: 2.3177 - val_accuracy: 0.0729
Epoch 3/5
3/3 [==============================] - 1s 357ms/step - loss: 2.2788 - accuracy: 0.1875 - val_loss: 2.3171 - val_accuracy: 0.0729
Epoch 4/5
3/3 [==============================] - 1s 319ms/step - loss: 2.2419 - accuracy: 0.1471 - val_loss: 2.3242 - val_accuracy: 0.0729
Epoch 5/5
3/3 [==============================] - 1s 369ms/step - loss: 2.2719 - accuracy: 0.1042 - val_loss: 2.3250 - val_accuracy: 0.0833


# Load Checkpoint

In [ ]:
model.load_weights('checkpoints')

# Saving Model

In [16]:
save_path = 'my_model.h5'

In [17]:
model.save(save_path, include_optimizer=True)

In [18]:
model = tf.keras.models.load_model('my_model.h5')

# Saving Model - 2

##### weight 저장

In [19]:
model.save_weights('model_weights.h5')

##### model 저장

In [20]:
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())

##### 구조 및 weight 불러오기

In [22]:
from tensorflow.keras.models import model_from_json

In [23]:
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

In [24]:
model.load_weights('model_weights.h5')

체크포인트가 있는 폴더를 지정해주면 체크포인트 불러오기 가능
>```python
model.load_weights('checkpoints')
```

# model.h5 들여다보기

In [25]:
import h5py

In [28]:
model_file = h5py.File('my_model.h5', 'r+')

In [29]:
model_file.keys()

<KeysViewHDF5 ['model_weights', 'optimizer_weights']>

In [30]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [31]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [33]:
model_file['model_weights']['conv2d']['conv2d']['kernel:0']

<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">

In [35]:
weight = np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])

In [36]:
weight

array([[[[-0.01790531, -0.09701243,  0.10658671, -0.03669837,
          -0.01511543, -0.00290571,  0.06228109,  0.05538224,
          -0.03237139,  0.1025598 ,  0.03816174,  0.00045612,
           0.02957754,  0.12038291,  0.01010264,  0.02087348,
          -0.00560233, -0.09366729,  0.07505775, -0.06445645,
           0.11932862, -0.0313741 ,  0.02101019, -0.10719249,
           0.05128941, -0.13271093,  0.04871363,  0.01158928,
          -0.00143963, -0.06459263,  0.02806435, -0.08604052],
         [ 0.04158574, -0.12016146,  0.12196834,  0.11362791,
           0.0654196 , -0.12018426, -0.09186264, -0.07875167,
           0.10266334, -0.06714372,  0.11110646,  0.0418493 ,
          -0.02933619,  0.11937815, -0.12383316, -0.03992603,
          -0.11837187, -0.0656889 , -0.08716273,  0.08039749,
          -0.07498445, -0.01239698,  0.01208303, -0.11107653,
          -0.06387948, -0.05412503, -0.04586476, -0.00024802,
           0.08990255,  0.01315073,  0.08083221,  0.07968722],
      